In [197]:
import pandas as pd
from collections import  Counter
from itertools import product, combinations

In [3]:
data = pd.read_csv('data.csv')
data.head(2)

,imdb_id,popularity,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year
0,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015
1,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015


In [4]:
data.columns

Index(['imdb_id', 'popularity', 'budget', 'revenue', 'original_title', 'cast',
       'director', 'tagline', 'overview', 'runtime', 'genres',
       'production_companies', 'release_date', 'vote_count', 'vote_average',
       'release_year'],
      dtype='object')

### ПРЕДОБРАБОТКА

In [5]:
columns_to_split = data.columns[data.apply(lambda x: x.str.contains('|',regex = False).any())]

In [20]:
data[columns_to_split] = data[columns_to_split].applymap(lambda x: x.split('|'))

**1. У какого фильма из списка самый большой бюджет?**

In [21]:
data.original_title.loc[data.budget.idxmax()]

"The Warrior's Way"

**2. Какой из фильмов самый длительный (в минутах)**

In [22]:
data.original_title.loc[data.runtime.idxmax()]

'Gods and Generals'

**3. Какой из фильмов самый короткий (в минутах)**

In [23]:
data.original_title.loc[data.runtime.idxmin()]

'Winnie the Pooh'

**4. Средняя длительность фильма?**

In [24]:
round(data.runtime.mean())

110

**5. Средняя длительность фильма по медиане?**

In [25]:
round(data.runtime.median())

106

**6. Какой самый прибыльный фильм?**

In [26]:
data['profit'] = data.revenue - data.budget

In [27]:
data.original_title.loc[data.profit.idxmax()]

'Avatar'

**7. Какой фильм самый убыточный?**

In [28]:
data.original_title.loc[data.profit.idxmin()]

"The Warrior's Way"

**8. Сколько всего фильмов в прибыли?**

In [29]:
len(data.loc[data.profit > 0])

1478

In [30]:
(data.profit>0).sum()

1478

**9. Самый прибыльный фильм в 2008 году?**

In [31]:
#year = pd.to_datetime(data.release_date).dt.year
data2008 = data[data.release_year == 2008]
data2008.original_title.loc[data2008.profit.idxmax()]

'The Dark Knight'

**10. Самый убыточный фильм за период с 2012 по 2014 (включительно)?**

In [32]:
data2012_2014 = data[data.release_year.isin([2012,2013,2014])]
data2012_2014.original_title.loc[data2012_2014.profit.idxmin()]

'The Lone Ranger'

**11. Какого жанра фильмов больше всего?**

In [33]:
total_genres_list = data.genres.sum()
pd.Series(total_genres_list).value_counts()[:1]

Drama    782
dtype: int64

In [34]:
Counter(total_genres_list).most_common(1)

[('Drama', 782)]

**12. Какого жанра среди прибыльных фильмов больше всего?**

In [35]:
profit_movie = data.loc[data.profit> 0]
total_genres_list = profit_movie.genres.sum()
pd.Series(total_genres_list).value_counts().index[0]

'Drama'

In [36]:
Counter(total_genres_list).most_common(1)[0][0]

'Drama'

**13. Кто из режиссеров снял больше всего фильмов?**

In [37]:
# data.director.value_counts()[:5] # поверхностное решение !!! после split не работает

In [38]:
%%time
directors_iter = Counter(data.director.sum())
dir_count = Counter()
for person, director in product(directors_iter,data.director):
    if person in director: dir_count[person] += 1
print(dir_count.most_common(1))

[('Steven Soderbergh', 13)]
Wall time: 388 ms


In [39]:
def groupby_listfeature(df, by):
    result = pd.DataFrame(columns=df.columns)
    next_index = 0
    to_replace = []
    for df_ind in df.index:
        for list_ind, value in enumerate(df[by][df_ind]):
            result.loc[next_index] = df.loc[df_ind]
            next_index = result.index.max()+1
            to_replace.append(value)
    result[by] = to_replace    
    return result.groupby(by)

In [40]:
%%time
groupby_listfeature(data[['director','imdb_id']],'director').count().idxmax()[0]
# А если после стирки не видно разницы, зачем платить больше? 

Wall time: 5 s


'Steven Soderbergh'

**14. Кто из режиссеров снял больше всего Прибыльных фильмов?**

In [41]:
profit_movie =  data.loc[data.profit > 0]





In [42]:
profit_movie =  data[['director','imdb_id']].loc[data.profit > 0]
groupby_listfeature(profit_movie,'director').count().idxmax()[0]

'Ridley Scott'

**15. Кто из режиссеров принес больше всего прибыли?**

In [43]:
%%time
directors_iter = Counter(data.director.sum())
profit_count = Counter()
for person in directors_iter:
    profit_per_director = (data.profit.loc[[person in directors for directors in data.director]] / 
                           data.director.loc[[person in directors for directors in data.director]].apply(len)).sum()
    profit_count[person] = profit_per_director
    
print(profit_count.most_common(1)[0][0])          # А если после стирки не видно разницы, зачем платить больше? 

Peter Jackson
Wall time: 1.84 s


In [44]:
%%time
data['profit_per_director'] = data.profit/data.director.apply(len)
print(groupby_listfeature(data[['director','profit_per_director']],'director').sum().idxmax()[0])

Peter Jackson
Wall time: 4.3 s


**16. Какой актер принес больше всего прибыли?**

In [45]:
actors = pd.Series(data.cast.sum()).unique()

In [46]:
%%time
actors_profit = pd.Series()
for actor in actors:
    actors_profit[actor] = data.profit[data.cast.str.contains(actor,regex=False)].sum()
print(actors_profit.sort_values(ascending = False).idxmax())

Emma Watson
Wall time: 8.87 s


In [47]:
%%time
actors_profit = Counter()
for actor in actors:
    actors_profit[actor] = data.profit[[actor in x for x in data.cast]].sum() #data.cast.str.contains(actor,regex=False)
print(actors_profit.most_common(1)[0][0])

Emma Watson
Wall time: 3.2 s


In [48]:
%%time
data['profit_per_actor'] = data.profit/data.cast.apply(len)
print(groupby_listfeature(data[['cast','profit_per_actor']] ,'cast' ).sum().idxmax()[0])

Emma Watson
Wall time: 22.5 s


**17. Какой актер принес меньше всего прибыли в 2012 году?**

In [49]:
%%time
data_2012 = data[data.release_year == 2012]
actors_2012 = pd.Series(data_2012.cast.sum()).unique()
actors_profit_2012 = Counter()
for actor in actors_2012:
    actors_profit_2012[actor] = data_2012.profit[[actor in x for x in data_2012.cast]].sum()
   
print(actors_profit_2012.most_common()[-1][0])

Kirsten Dunst
Wall time: 201 ms


In [50]:
%%time
groupby_listfeature(data_2012[['cast','profit_per_actor']],'cast').sum().idxmin()[0]

Wall time: 1.26 s


'Kirsten Dunst'

**18. Какой актер снялся в большем количестве высокобюджетных фильмов? (в фильмах где бюджет выше среднего по данной выборке)**

In [51]:
%%time
high_budget_data = data[data.budget > data.budget.mean()]
total_actors_list = high_budget_data.cast.sum() 
print(pd.Series(total_actors_list).value_counts().idxmax())

Matt Damon
Wall time: 13 ms


In [52]:
%%time
print(groupby_listfeature(high_budget_data[['cast','imdb_id']],'cast').count().idxmax()[0])

Matt Damon
Wall time: 7.07 s


**19. В фильмах какого жанра больше всего снимался Nicolas Cage?**

In [53]:
%%time
print(pd.Series(data[['Nicolas Cage' in x for x in data.cast]].genres.sum()).value_counts().idxmax())

Action
Wall time: 5 ms


In [54]:
%%time
data_NicCage = data[['Nicolas Cage' in x for x in data.cast]]
print(groupby_listfeature(data_NicCage[['imdb_id','genres']],'genres').count().idxmax()[0])

Action
Wall time: 259 ms


**20. Какая студия сняла больше всего фильмов?**

In [55]:
%%time
print(pd.Series(data.production_companies.sum()).value_counts().idxmax())

Universal Pictures
Wall time: 47 ms


In [56]:
%%time
print(Counter((data.production_companies.sum())).most_common(1)[0][0])

Universal Pictures
Wall time: 57 ms


**21. Какая студия сняла больше всего фильмов в 2015 году?**

In [57]:
data_2015 = data[data.release_year == 2015]
print(pd.Series(data_2015.production_companies.sum()).value_counts().idxmax())

Warner Bros.


**22. Какая студия заработала больше всего денег в жанре комедий за все время?**

In [58]:
data['profit_per_pc'] = data.profit / data.production_companies.apply(len)
comedy = data[['Comedy' in x for x in data.genres]]
production_companies_list = pd.Series(comedy.production_companies.sum()).unique()

In [59]:
prod_comp_profit = pd.Series()
for prod_comp in production_companies_list:
    prod_comp_profit[prod_comp] = comedy.profit[[prod_comp in x for x in comedy.production_companies]].sum()
prod_comp_profit.idxmax()

'Universal Pictures'

In [60]:
groupby_listfeature(comedy[['production_companies','profit']] ,'production_companies').sum().idxmax()[0]

'Universal Pictures'

In [61]:
prod_comp_profit = pd.Series()
for prod_comp in production_companies_list:
    prod_comp_profit[prod_comp] = comedy.profit_per_pc[[prod_comp in x for x in comedy.production_companies]].sum()
prod_comp_profit.idxmax()

'Walt Disney Pictures'

In [62]:
groupby_listfeature(comedy[['production_companies','profit_per_pc']] ,'production_companies').sum().idxmax()[0]

'Walt Disney Pictures'

**23. Какая студия заработала больше всего денег в 2012 году?**

In [63]:
data_2012 = data[data.release_year == 2012]
production_companies_list = pd.Series(data_2012.production_companies.sum()).unique()

In [64]:
pr_comp_profit = pd.Series()
for pr_comp in production_companies_list:
    pr_comp_profit[pr_comp] = data_2012.profit[[pr_comp in x for x in data_2012.production_companies]].sum()
pr_comp_profit.idxmax()

'Columbia Pictures'

In [65]:
groupby_listfeature(data_2012[['production_companies','profit_per_pc']], 'production_companies').sum().idxmax()[0]

'Columbia Pictures'

**24. Самый убыточный фильм от Paramount Pictures**

In [66]:
data_ParPic = data[['Paramount Pictures' in x for x in data.production_companies]]
data_ParPic.loc[data_ParPic.profit.idxmin()].original_title

'K-19: The Widowmaker'

**25. Какой Самый прибыльный год (заработали больше всего)?**

In [67]:
data.groupby('release_year').profit.sum().idxmax()

2015

**26. Какой Самый прибыльный год для студии Warner Bros?**

In [68]:
prod_comp_list = pd.Series(data.production_companies.sum()).unique()
prod_comp_list[['Warner' in x for x in prod_comp_list]]

array(['Warner Bros.', 'Warner Bros. Animation',
       'Warner Bros. Interactive Entertainment', 'Warner Bros. Pictures',
       'Warner Independent Pictures (WIP)'], dtype=object)

In [69]:
data_WarBros = data[['Warner Bros.' in x for x in data.production_companies]]

In [70]:
data_WarBros.groupby('release_year').profit.sum().idxmax()

2007

In [71]:
data_WarBros.groupby('release_year').profit_per_pc.sum().idxmax()

2010

In [72]:
data_WarBros = data[[pd.Series(['Warne' in y for y in x]).any() for x in data.production_companies]]

In [73]:
data_WarBros.groupby('release_year').profit.sum().idxmax()

2014

In [74]:
data_WarBros.groupby('release_year').profit_per_pc.sum().idxmax()

2010

**27. В каком месяце за все годы суммарно вышло больше всего фильмов?**

In [75]:
data['release_month'] = pd.to_datetime(data.release_date).dt.month

In [76]:
data.groupby('release_month').imdb_id.count().idxmax()

9

In [77]:
data.groupby(['release_month','release_year']).imdb_id.count().idxmax()

(9, 2009)

**28. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)**

In [78]:
data [[x in [6,7,8] for x in data.release_month]].imdb_id.count()

450

**29. Какой режисер выпускает (суммарно по годам) больше всего фильмов зимой?**

In [79]:
%%time
data_winter = data.query('release_month in [1,2,12]')

Wall time: 9 ms


In [80]:
%%time
data_winter = data [[x in [1,2,12] for x in data.release_month]]

Wall time: 2 ms


In [81]:
groupby_listfeature(data_winter[['director','imdb_id']], 'director').count().idxmax()[0]

'Peter Jackson'

**30. Какой месяц чаще всего по годам самый прибыльный?**

In [82]:
pivtab_year_month = data.pivot_table('profit', index= 'release_year', columns= 'release_month', aggfunc= sum)
pivtab_year_month.idxmax(axis=1).max()

12

**31. Названия фильмов какой студии в среднем самые длинные по количеству символов?**

In [83]:
gruoppedby_pc = groupby_listfeature(data[['production_companies','original_title']],'production_companies')
gruoppedby_pc.original_title.apply(lambda x: len(x.sum())/x.count()).idxmax()

'Four By Two Productions'

**32. Названия фильмов какой студии в среднем самые длинные по количеству слов?**

In [84]:
#gruoppedby_pc.original_title.apply(lambda x: x.append(x)[1:])

In [85]:
gruoppedby_pc.original_title.apply(lambda x: 
                                   len(((x+" ").sum()).split(' '))
                                   /x.count()).idxmax()

'Four By Two Productions'

**33. Сколько разных слов используется в названиях фильмов?(без учета регистра)**

In [86]:
import re

In [87]:
total_title_list = data.original_title.apply(lambda x: x+' ').sum().lower()

In [88]:
pattern = re.compile('(?![-])\S*')

In [90]:
len(pd.Series(pattern.findall(total_title_list)).unique())

2461

**34. Какие фильмы входят в 1 процент лучших по рейтингу?**

Варианты ответа:

1. Inside Out, Gone Girl, 12 Years a Slave


In [91]:
data.original_title.loc[data.vote_average.sort_values(ascending = False)[:int(0.01*(len(data)))].index].sort_index()

9                                          Inside Out
28                                          Spotlight
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
177                            The Fault in Our Stars
283                                        Mr. Nobody
370                                         Inception
600                                   The Dark Knight
864             The Lord of the Rings: The Two Towers
873                                       The Pianist
1082    The Lord of the Rings: The Return of the King
1184                          The Wolf of Wall Street
1192                                 12 Years a Slave
1801                                          Memento
Name: original_title, dtype:

In [92]:
import numpy as np
percent_99 = np.percentile(data.vote_average,99)
data.original_title[data.vote_average > percent_99]

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
600                                   The Dark Knight
873                                       The Pianist
1082    The Lord of the Rings: The Return of the King
1184                          The Wolf of Wall Street
1192                                 12 Years a Slave
1801                                          Memento
Name: original_title, dtype: object

**35. Какие актеры чаще всего снимаются в одном фильме вместе**

In [194]:
from itertools import combinations

In [195]:
colleagues_count  = Counter()
for cast in data.cast:
    for one, another in list(combinations(cast, 2)):
        colleagues_count[sorted([one,another])[0] +' + ' + sorted([one,another])[1]] += 1

In [200]:
colleagues_count.most_common(1)

[('Daniel Radcliffe + Rupert Grint', 8)]

**36. У какого из режиссеров выше вероятность выпустить фильм в прибыли?  
У какого из режиссеров самый высокий процент фильмов со сборами выше бюджета?**

In [203]:
grouppedby_director = groupby_listfeature(data[['director','profit']],'director')

In [225]:
grouppedby_director.apply(lambda x:(x.profit > 0).sum()/x.profit.count())[
['Quentin Tarantino',
'Steven Soderbergh',
'Robert Rodriguez',
'Christopher Nolan',
'Clint Eastwood']].idxmax()

'Christopher Nolan'

In [187]:
# -------------------------------ПОДВАЛ-----------------------------------------

import itertools

lst = [[1, 2, 3], [4], [5, 6, 7]]  
list(itertools.chain.from_iterable(lst))

[1, 2, 3, 4, 5, 6, 7]


In [ ]:
def features_list_splitting(df, feature):
    result = pd.DataFrame(columns=df.columns)
    feature_column = df.columns.tolist().index(feature)
    next_index = 0
    
    for df_ind in df.index:
        for list_ind, feature_value in enumerate(df.loc[df_ind][feature]):
            row = df.loc[df_ind].values
            row[feature_column] = row[feature_column][list_ind]
            result.loc[next_index] = row
            next_index = result.index.max()+1
    return result

In [ ]:
def groupby_listfeature: (data, by):
    result = pd.DataFrame(columns=df.columns)
    next_index = 0
    to_replace = []
    for df_ind in df.index:
        for list_ind, value in enumerate(df[by][df_ind]):
            result.loc[next_index] = df.loc[df_ind]
            next_index = result.index.max()+1
            to_replace.append(value)
    result[by] = to_replace    
    return result.groupby(by)

In [160]:
df = data.copy()
by = 'director'

result = pd.DataFrame(columns=df.columns)

next_index = 0
to_replace = []
for df_ind in df.index:
    for list_ind, value in enumerate(df[by][df_ind]):
        result.loc[next_index] = df.loc[df_ind]
        next_index = result.index.max()+1
        to_replace.append(value)
result[by] = to_replace

In [161]:
result.groupby(by)['profit_per_director'].sum().idxmax()

'Peter Jackson'

In [ ]:
result.groupby(by)

In [157]:
directors_iter = Counter(data.director.sum())

In [161]:
%%time
dir_count = Counter()
for person, director in product(directors_iter,data.director):
    if person in director: dir_count[person] += 1
print(dir_count.most_common(1))

[('Steven Soderbergh', 13)]
Wall time: 372 ms


In [160]:
%%time
directors_count = pd.Series()
for director in directors_iter:
    directors_count[director] = data.director[data.director.str.contains(director,regex = False)].count()
print(directors_count.sort_values(ascending = False)[:1]) #.idxmax()

Steven Soderbergh    13
dtype: int64
Wall time: 2.26 s


In [162]:
%%time
directors_count = pd.Series()
for person in directors_iter:
    directors_count[person] = data.director[[person in director for director in data.director]].count()
print(directors_count.sort_values(ascending = False)[:1]) #.idxmax()

Steven Soderbergh    13
dtype: int64
Wall time: 1.8 s


In [ ]:
%%time
actors_profit = pd.Series()
for actor in actors:
    actors_profit[actor] = data.profit[data.cast.str.contains(actor,regex=False)].sum()

print(actors_profit.sort_values(ascending = False)[:5]) #.idxmax()

In [72]:
%%time
directors_iter = Counter(data.director.sum())
profit_count = Counter()
for person in directors_iter:
    profit_per_director = (data.profit.loc[[person in directors for directors in data.director]] / 
                           data.director.loc[[person in directors for directors in data.director]].apply(len)).sum()
    profit_count[person] = profit_per_director
print(profit_count.most_common(1))

[('Peter Jackson', 5202593685.0)]
Wall time: 1.37 s


In [62]:
person = 'Lilly Wachowski'
(data.profit.loc[[person in directors for directors in data.director]] / 
data.director.loc[[person in directors for directors in data.director]].apply(len)).sum()

432254988.3333334

In [59]:
data.director.loc[[person in d for d in data.director]].apply(len)

11      2
669     2
994     3
1086    2
1087    2
Name: director, dtype: int64

In [47]:
data.director[:13]

0                     [Colin Trevorrow]
1                       [George Miller]
2                    [Robert Schwentke]
3                         [J.J. Abrams]
4                           [James Wan]
5      [Alejandro GonzÃ¡lez IÃ±Ã¡rritu]
6                         [Alan Taylor]
7                        [Ridley Scott]
8           [Kyle Balda, Pierre Coffin]
9                         [Pete Docter]
10                         [Sam Mendes]
11    [Lana Wachowski, Lilly Wachowski]
12                       [Alex Garland]
Name: director, dtype: object

In [139]:
lst = [[1, 2, 3], [4], [5, 6, 7]]

In [143]:
pd.Series(lst).sum()

[1, 2, 3, 4, 5, 6, 7]

In [28]:
import itertools

In [29]:
lst = [[1, 2, 3], [4], [5, 6, 7]]  
list(itertools.chain.from_iterable(lst))

[1, 2, 3, 4, 5, 6, 7]

In [53]:
data.index.max()

1889

In [57]:
#data[data.director.str.contains('|')]
# ord("c")
#
# chr(99)

In [81]:
ord(data.director[8][10])

124

In [82]:
chr(124)

'|'

In [84]:
data.director[8][10].find(chr(124))

0

In [86]:
ord(data.director[8][0])

75

In [91]:
data.director.apply(lambda x: x.split('|'))[:9]

0                   [Colin Trevorrow]
1                     [George Miller]
2                  [Robert Schwentke]
3                       [J.J. Abrams]
4                         [James Wan]
5    [Alejandro GonzÃ¡lez IÃ±Ã¡rritu]
6                       [Alan Taylor]
7                      [Ridley Scott]
8         [Kyle Balda, Pierre Coffin]
Name: director, dtype: object

In [272]:
data2 = data.copy()

In [273]:
data2.director = data2.director.apply(lambda x: x.split('|'))

In [274]:
data2.cast = data2.cast.apply(lambda x: x.split('|'))

In [275]:
ttt = pd.DataFrame(columns=data.columns)
ttt

,imdb_id,popularity,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,profit


In [276]:
ttt = pd.DataFrame(columns=data.columns)


director_column = ttt.columns.tolist().index('director')
cast_column = ttt.columns.tolist().index('cast')
next_index = 0

for i in data2.index: #.loc[:12]
    for j,director in enumerate(data2.loc[i].director):
        row = data2.loc[i].values
        row[director_column] = row[director_column][j]
        ttt.loc[next_index] = row
        next_index = ttt.index.max()+1
        


In [277]:
ttt

,imdb_id,popularity,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,profit
0,tt0369610,32.985763,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,1363528810
1,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,228436354
2,tt2908446,13.112507,110000000,295238201,Insurgent,"[Shailene Woodley, Theo James, Kate Winslet, A...",Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,185238201
3,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...",J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,1868178225
4,tt2820852,9.335014,190000000,1506249360,Furious 7,"[Vin Diesel, Paul Walker, Jason Statham, Miche...",James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,1316249360
5,tt1663202,9.110700,135000000,532950503,The Revenant,"[Leonardo DiCaprio, Tom Hardy, Will Poulter, D...",Alejandro GonzÃ¡lez IÃ±Ã¡rritu,"(n. One who has returned, as if from the dead.)","In the 1820s, a frontiersman, Hugh Glass, sets...",156,Western|Drama|Adventure|Thriller,Regency Enterprises|Appian Way|CatchPlay|Anony...,12/25/2015,3929,7.2,2015,397950503
6,tt1340138,8.654359,155000000,440603537,Terminator Genisys,"[Arnold Schwarzenegger, Jason Clarke, Emilia C...",Alan Taylor,Reset the future,"The year is 2029. John Connor, leader of the r...",125,Science Fiction|Action|Thriller|Adventure,Paramount Pictures|Skydance Productions,6/23/2015,2598,5.8,2015,285603537
7,tt3659388,7.667400,108000000,595380321,The Martian,"[Matt Damon, Jessica Chastain, Kristen Wiig, J...",Ridley Scott,Bring Him Home,"During a manned mission to Mars, Astronaut Mar...",141,Drama|Adventure|Science Fiction,Twentieth Century Fox Film Corporation|Scott F...,9/30/2015,4572,7.6,2015,487380321
8,tt2293640,7.404165,74000000,1156730962,Minions,"[Sandra Bullock, Jon Hamm, Michael Keaton, All...",Kyle Balda,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",91,Family|Animation|Adventure|Comedy,Universal Pictures|Illumination Entertainment,6/17/2015,2893,6.5,2015,1082730962
9,tt2293640,7.404165,74000000,1156730962,Minions,"[Sandra Bullock, Jon Hamm, Michael Keaton, All...",Pierre Coffin,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",91,Family|Animation|Adventure|Comedy,Universal Pictures|Illumination Entertainment,6/17/2015,2893,6.5,2015,1082730962


In [270]:
# ttt.groupby('director')['profit'].sum().sort_values(ascending = False)

In [253]:
def features_list_splitting(df, feature):
    """
    Принимает df в котором колонка feature содержит list значений . 
    Для каждой строки в df количество элементов в list различно.
    
    Возвращает pd.DataFrame в котором каждой i-ой строке из df соответсвует 
    n строк, где n - количество элементов в list у i-ой строки.
    При этом колонка feature принимает последовательно значения из list, остальные колонки 
    совпадают с колонками из i-ой строки df.
    """
    result = pd.DataFrame(columns=df.columns)
    feature_column = df.columns.tolist().index(feature)
    next_index = 0
    
    for df_ind in df.index:
        for list_ind, feature_value in enumerate(df.loc[df_ind][feature]):
            row = df.loc[df_ind].values
            row[feature_column] = row[feature_column][list_ind]
            result.loc[next_index] = row
            next_index = result.index.max()+1
    return result

In [282]:
%%time
data_tr_spl = features_list_splitting(data2, 'director')

Wall time: 17.4 s


In [284]:
%%time
data_tr_spl = features_list_splitting(data_tr_spl, 'cast')

Wall time: 2min 34s


In [2]:
vehicles = [('Ford', 'Taurus'), ('Dodge', 'Durango'),
            ('Chevrolet', 'Cobalt'), ('Ford', 'F150'),
            ('Dodge', 'Charger'), ('Ford', 'GT')]
 
sorted_vehicles = sorted(vehicles)
sorted_vehicles

[('Chevrolet', 'Cobalt'),
 ('Dodge', 'Charger'),
 ('Dodge', 'Durango'),
 ('Ford', 'F150'),
 ('Ford', 'GT'),
 ('Ford', 'Taurus')]